In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from time import sleep
from datetime import datetime, timedelta
import string
import csv
from bs4 import BeautifulSoup

In [2]:

class WorldometersCrawl:
    def __init__(self):
        self.service = Service(".\chromedriver.exe")
        self.driver = webdriver.Chrome(service=self.service, options=self.driver_option())
        self.driver.implicitly_wait(5)
        
    def driver_option(self):
        options = webdriver.ChromeOptions()
        #options.add_argument("--headless")
        options.add_argument("--window-size=2000x2000")
        options.add_argument("start-maximized")
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--blink-settings=imagesEnabled=false')
        options.add_argument('disable-infobars')
        options.add_argument("--verbose")
        options.add_argument("--no-default-browser-check")
        options.add_argument("--ignore-ssl-errors")
        options.add_argument("--allow-running-insecure-content")
        options.add_argument("--disable-web-security")
        options.add_argument("--disable-feature=IsolateOrigins,site-per-process")
        options.add_argument("--no-first-run")
        options.add_argument("--mute-audio")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-translate")
        options.add_argument("--ignore-certificate-error-spki-list")
        options.add_argument("--ignore-certificate-errors")
        options.add_argument("--disable-blink-features=AutomationControllered")
        options.add_experimental_option('useAutomationExtension', False)
        prefs = {"profile.default_content_setting_values.notifications": 2}
        options.add_experimental_option("prefs", prefs)
        # overcome limited resource problems
        options.add_argument("--disable-dev-shm-usage")
        options.add_experimental_option(
            "excludeSwitches", ["enable-automation"])
        options.add_experimental_option(
            "prefs", {"profile.managed_default_content_settings.images": 2})
        options.add_argument("user-agent=foo")
        return options
    
    def open_link(self, link):
        self.driver.get(link)
        
    def main(self):
        self.open_link('https://www.worldometers.info/coronavirus/')
        data = self.crawl_data()
        csvfile = str((datetime.today() - timedelta(days=2)).strftime("%d-%m-%Y")) + " raw.csv"
        print(csvfile)
        self.write_data(csvfile, data)
        
    def crawl_data(self):
        data = []
        url = self.driver.current_url
        
        html_source = self.driver.page_source 
        soup = BeautifulSoup(html_source, 'html.parser')
        
        elements = soup.find('table',{'id':'main_table_countries_yesterday2'}).find_all('tr', {'role': 'row'})
        for i in range(1, len(elements)):
            el = elements[i].find_all('td')
            data.append([el[i].text.strip() for i in range(1,16)])
        return data

    def write_data(self, file, data):
        header = ['Country, Other','Total cases','New cases','Total deaths','New deaths',
                  'Total recovered','New recovered','Active cases', 'Serious, Critical', 'Tot Cases/1M pop', 'Deaths/1M pop',
                  'Total tests','Test/1M pop','Population', 'Continent']
        with open(file, 'w', encoding='UTF8', newline='') as csvfile:
            writer = csv.writer(csvfile, quoting=csv.QUOTE_NONNUMERIC, delimiter=',')
            writer.writerow(header) # Write header 
            for line in data:
                writer.writerow(line) # Write user information into file

In [3]:
worldometers = WorldometersCrawl()
worldometers.main()

04-12-2021 raw.csv
